## MNIST Classifier

Classifies handwritten digits (0-9)<br>
Last update: 1/1/24

In [3]:
import cupy as np
from importlib import reload
from keras.datasets import mnist
from PIL import Image
from sandbox import activations, costs, initializers, layers, model, optimizers, utils

In [4]:
# Inform utils that we're using CuPy instead of NumPy
utils.configure_imports(cuda=True)

# Load data
(train_x, train_y), (test_x, test_y) = mnist.load_data()

# Flatten, normalize, and reshape data for model
train_x = np.asarray(train_x.reshape(train_x.shape[0], -1) / 255)
test_x = np.asarray(test_x.reshape(test_x.shape[0], -1) / 255)
train_y = utils.one_hot(train_y, num_classes=10)
test_y = utils.one_hot(test_y, num_classes=10)

In [5]:
# Create model
digits = model.Model(cuda=True)
digits.add(layers.Dense(units=32, activation=activations.ReLU()))
digits.add(layers.Dense(units=16, activation=activations.ReLU()))
digits.add(layers.Dense(units=train_y.shape[1], activation=activations.Softmax()))

# Configure model
digits.configure(
    input_size=train_x.shape[1],
    optimizer=optimizers.Adam(),
    cost_type=costs.CategoricalCrossentropy(),
)

digits.train(train_x, train_y, epochs=3, batch_size=64, verbose=True)

Cost on epoch 1: 0.1551
Cost on epoch 2: 0.0828
Cost on epoch 3: 0.11734


In [6]:
reload(utils)

# Print model summary
digits.summary()

# Assess model accuracy
pred_train = digits.predict(train_x)
print('\nTraining Accuracy:', utils.evaluate(pred_train, train_y))
pred_test = digits.predict(test_x)
print('Testing Accuracy:', utils.evaluate(pred_test, test_y))

# Use model on custom image
seven = Image.open('seven.png').convert('L')
seven = np.array(seven).reshape(1, -1) / 255
pred_seven = digits.predict(seven)
print('Custom Image Prediction:', np.argmax(pred_seven))

+------------+------------+
| Layer type | Parameters |
+------------+------------+
|   Dense    |   25120    |
|   Dense    |    528     |
|   Dense    |    170     |
+------------+------------+
Total parameters: 25818

Training Accuracy: 0.9658833333333333
Testing Accuracy: 0.957
Custom Image Prediction: 7
